In [1]:
# import all libs (do not change)
from pymongo import MongoClient
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import json
from bson import Code
import pymongo
import pprint
import nltk
from nltk.corpus import stopwords
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from itertools import combinations
from collections import defaultdict
nltk.download('stopwords')
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 29.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# fill in uri
uri = '<Fill in with Mongo Atlas ip, username, password>'
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    capture = client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!", capture)
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB! {'ok': 1}


In [3]:
# database (do not change)
db = client['test']
posts = db['litcovidtest']

In [4]:
# transaction preparation
# first need to query the database to obtain all 'passages.text'
# fill in (10pts)
result = posts.find({})



stops = set(stopwords.words('english'))
trns = []
for post in result:
# get each paper
    for text in post['passages']:
        #get each paragraph in passages
        t=text['text']
        trn = set()
        # seperate sentences
        sentences = nltk.sent_tokenize(t)
        for sent in sentences:
            # seperate words
            tokens = nltk.word_tokenize(sent)
            for token in tokens:
                # convert all words to lower case
                token=token.lower()
                # remove stopwords, punctuations, and single char
                if token not in stops and len(token)>1 :
                     trn.add(token)
        trns.append(list(trn))
# trns is the list of transactions

print("count of transactions:", len(trns))

count of transactions: 35590


In [ ]:
# functions to mine patterns and their frequency
# fill in function
def frequentItem(transactions, minSupport, k):
    transaction_sets = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            transaction_sets[item] += 1
    transaction_sets = dict((item, support) for item, support in transaction_sets.items() if support >= minSupport)
    frequency_sets = transaction_sets.copy()
    if k == 1:
        return sorted(frequency_sets.items(), key=lambda x: x[1], reverse=True)
    elif k == 2:
        pairs = defaultdict(int)
        for transaction in transactions:
            for i in range(len(transaction)):
                    for j in range(i+1, len(transaction)):
                        if transaction[i] in transaction_sets and transaction[j] in transaction_sets:
                            pair = ','.join(sorted([transaction[i], transaction[j]]))
                            pairs[pair] += 1
        pairs = {pair:count for pair, count in pairs.items() if count >= minSupport}
        return sorted(pairs.items(), key=lambda x: x[1], reverse=True)
    else:
        for i in range(2, k+1):
            frequency_sets = defaultdict(int)
            for itemset in combinations(transaction_sets, i):
                support = min(transaction_sets[item] for item in itemset)
                frequency_sets[itemset] = support
            frequency_sets = dict((itemset, support) for itemset, support in frequency_sets.items() if support >= minSupport)
            transaction_sets.update(frequency_sets)
        return sorted(frequency_sets.items(), key=lambda x: x[1], reverse=True)


: 

In [6]:
# find all frequent 1-itemset with minSupport = 200
# 20pts
f1item=frequentItem(trns, minSupport=200, k=1)
countF1item = len(f1item)
print("count of frequent 1-itemset with minSupport=200:", countF1item)
pprint.pprint(f1item[0:5])

count of frequent 1-itemset with minSupport=200: 758
[('covid-19', 9053),
 ('patients', 4704),
 ('study', 3213),
 ('sars-cov-2', 2909),
 ('disease', 2843)]


In [7]:
# find all frequent 1-itemset with minSupport = 500
# 10pts
f1item=frequentItem(trns, minSupport=500, k=1)
countF1item = len(f1item)
print("count of frequent 1-itemset with minSupport=500:", countF1item)

count of frequent 1-itemset with minSupport=500: 205


In [8]:
# find all frequent 1-itemset with minSupport = 1000
# 10pts
f1item=frequentItem(trns, minSupport=1000, k=1)
countF1item = len(f1item)
print("count of frequent 1-itemset with minSupport=1000:", countF1item)

count of frequent 1-itemset with minSupport=1000: 58


In [9]:
# find all frequent 2-itemset with minSupport = 200
# 20pts
f2item=frequentItem(trns, minSupport=200, k=2)
countF2item = len(f2item)
print("count of frequent 2-itemset with minSupport=200:", countF2item)
pprint.pprint(f2item[0:5])

count of frequent 2-itemset with minSupport=200: 286903
[(('covid-19', 'patients'), 4704),
 (('covid-19', 'study'), 3213),
 (('patients', 'study'), 3213),
 (('covid-19', 'sars-cov-2'), 2909),
 (('patients', 'sars-cov-2'), 2909)]


In [10]:
# find all frequent 2-itemset with minSupport = 400
# 10pts
f2item=frequentItem(trns, minSupport=400, k=2)
countF2item = len(f2item)
print("count of frequent 2-itemset with minSupport=400:", countF2item)

count of frequent 2-itemset with minSupport=400: 40755


In [11]:
# find all frequent 2-itemset with minSupport = 600
# 10pts
f2item=frequentItem(trns, minSupport=600, k=2)
countF2item = len(f2item)
print("count of frequent 2-itemset with minSupport=600:", countF2item)

count of frequent 2-itemset with minSupport=600: 10585


In [12]:
# find all frequent 2-itemset with minSupport = 800
# 10pts
f2item=frequentItem(trns, minSupport=800, k=2)
countF2item = len(f2item)
print("count of frequent 2-itemset with minSupport=800:", countF2item)

count of frequent 2-itemset with minSupport=800: 3828
